In [ ]:
"""
CONTRIBUTORS:
    Paul Durham
FILE CONTENT DESCRIPTION:
	The file main.py leverages functionality from the various components of SLISH in order to produce a working, efficient 
	software. The user interface design and functionalities are implemented utilizing the tkinter library for its development.
	As previously mentioned, the methods from every component of SLISH are used logically in order for SLISH to act accordingly
	based on the recevied input. 
	
	Specific methods and logic leveraged from the camera component in this file include the system 
	"cool down" period and retrieving the external input for use by the rest of the system.
	
	The classifier comoponent's methods are used frequently throughout the latter contents of this file.
    Based on the classifications made by the classifier component, logic in main.py determines whether enough
	frames have been classified to make an accurate gesture prediction and, if so, whether to execute a command 
	based on the received input. Specifically, this ensures that at least 6 frames were analyzed before a prediction
	is made, ensures that commands are only recognized in a format valid to SLISH (letter -> number) and handles
	invalid or unrecognized input without affecting system performance. 
	Last, methods from the socket class are utilized to control the web sockets that correspond to the command received.
	The actions that are executed are clearly displayed within the contents of the user interface. 
REQUIREMENTS ADDRESSED:
    FR.2, FR.2.1, FR 2.1.1, FR.2.1.2, FR.2.1.3, FR2.2, FR.2.3,
	NFR.2, NFR.4, NFR.5, NFR.6, NFR.7, NFR.8, NFR.9
	EIR.1, EIR.2
LICENSE INFORMATION:
    Copyright (c) 2019, CSC 450 Group 1
    All rights reserved.
    Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
    following conditions are met:
        * Redistributions of source code must retain the above copyright notice, this list of conditions and the
          following disclaimer.
        * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and
          the following disclaimer in the documentation and/or other materials provided with the distribution.
        * Neither the name of the CSC 450 Group 4 nor the names of its contributors may be used to endorse or
          promote products derived from this software without specific prior written permission.
    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,
    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL <COPYRIGHT HOLDER> BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY,
    OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
    DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT,
    STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE,
    EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

from __future__ import print_function 
from __future__ import division
import torch as torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


# Global Variables init
input_size = 0
device = 'cpu'
num_classes = 6

## This function will take a model and train the model.
def train(model):

    model_name = "alexnet"

    ## This is the batch size for the training. This can be changed based on the amount of ram your system has.
    batch_size = 8

    ## The number of epochs to train for
    num_epochs = 15

    ## feature_extract = True = The reshaped layer params will be updated 
    ## feature_etract = False = Finetune the whole model
    feature_extract = True

    ## Init the model incase we are not bringing a model in
    model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

    ## A transformer that will reshape/change the incoming images
    data_transforms = {
        'train': transforms.Compose([
            ## This will add some randomization to the images
            #transforms.RandomAffine(degrees=10, translate=[0,.3], scale=[.001,.3], shear=None, resample=False, fillcolor=0),
            transforms.Resize(input_size),
            transforms.Grayscale(num_output_channels=3),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.Grayscale(num_output_channels=3),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    ## Create data loaders for the images
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    ## Check if a gpu is available for calculations
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    ## Send the model to the device (cpu/gpu)
    model_ft = model_ft.to(device)

    ## Gets the parameters from the model to be trained 
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

    ## sets the lossfunction  crossentropyloss
    criterion = nn.CrossEntropyLoss()

    ## begin training and evaluation
    if model != None:
        print("Trained the loaded model")
        model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))
    else:
        print("Trained a new model")
        model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))    
    ## Saves model
    torch.save(model_ft.state_dict(),'model.pt')

## Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

## Sets parameters that require grad
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=False):
    model_ft = None
    
    ## This init function is only able to init alexnet model.
    if odel_name == "alexnet":

        model_ft = torch.hub.load('pytorch/vision:v0.5.0', 'alexnet', pretrained=True)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    return model_ft, input_size

## Loads model
## Change model.pt to the model name you wish to load
def load_model():
    return torch.load('../model_handler/model.pt')
    


model = None

try:
    model = load_model()
except Exception:
    print("Creating new model")
train(model)